In [223]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [224]:
import numpy as np
from scipy.optimize import minimize
import networkx as nx
from pathlib import Path
from scipy.special import logsumexp
from scipy.optimize import minimize, OptimizeResult

# Import functions for loading TNTP data
from src.load_data import (
    read_graph_transport_networks_tntp,
    read_traffic_mat_transport_networks_tntp,
)

from src.models import BeckmannModel

In [225]:
# x = np.array([1, 2, 2, 2])
# softmin(x, beta=20)

In [226]:
# def softmin(x, beta):
#     """ Compute the smooth minimum of an array x with smoothing parameter beta """
# #     max_x = np.max(x)
# #     x_stable = x - max_x
# #     weights = np.exp(-beta * x_stable)
# #     return np.dot(weights, x) / np.sum(weights)
#     return -logsumexp(-beta * x) / beta
# #     return -logsumexp(-x)

# # # Function to compute sigma_star, which is the sum of squares of the elements
# # def sigma_star(t):
# #     """ Compute the sum of squares of the elements of the array t """
# #     return (t ** 2).sum()

def numerical_gradient(f, x, eps=1e-8):
    # Set up an array to store our gradient calculations.
    grad = np.zeros_like(x)
    
    # Let's go through each element in x to find the gradients.
    for i in range(len(x)):
        x_plus = x.copy()
        x_plus[i] += eps  # Nudge the element up a bit by eps.
        
        x_minus = x.copy()
        x_minus[i] -= eps  # Now nudge it down by eps and see what happens.
        
        # Calculate the gradient using the central difference formula.
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * eps)
    
    return grad


def softmin(x, beta):
    """ Compute the smooth minimum of an array x with smoothing parameter beta """
    return -logsumexp(-beta * x) / beta


# Define the dual objective function using the softmin
def dual_objective(y_flattened, A, L, beta):
    """ The dual objective function to be minimized """
    y = y_flattened.reshape((nodes, nodes))
    ATy = A.T @ y
    
    # Применяем softmin к каждой строке ATy
    softmin_values = np.array([softmin(ATy_row, beta) for ATy_row in ATy])
    
    # Вычисляем значение целевой функции
    objective_value = np.sum(beckmann_model.sigma_star(-softmin_values)) - np.sum(y * L)
    
    if np.isnan(objective_value) or np.isinf(objective_value):
        print("NaN or Inf detected in objective function!")
        return np.finfo(float).max
    
    return objective_value

def dual_objective_grad(y_flattened, A, L, beta):
    """ Gradient of the target function of the dual problem """
    y = y_flattened.reshape((nodes, nodes))
    ATy = A.T @ y
    
    # Calculate softmin for each line of ATy
    softmin_values = np.array([softmin(ATy_row, beta) for ATy_row in ATy])
    
    # Calculate the derivative of softmin
    softmin_grad = np.exp(-beta * (ATy - softmin_values[:, np.newaxis]))
    softmin_grad /= np.sum(softmin_grad, axis=1, keepdims=True)
    
    # Calculate the sigma_star gradient numerically
    sigma_star_grad = numerical_gradient(lambda x: np.sum(beckmann_model.sigma_star(x)), -softmin_values)
    
    # Calculate the y gradient
    grad = -A @ (sigma_star_grad[:, np.newaxis] * softmin_grad) - L
    
    grad_flattened = grad.flatten()
    
    if np.isnan(grad_flattened).any() or np.isinf(grad_flattened).any():
        print("NaN or Inf detected in gradient!")
        return np.zeros_like(grad_flattened)
    
    return grad_flattened

def scaled_minimize(fun, x0, args=(), jac=None, **kwargs):
    # Determine the scale based on the maximum absolute value of initial guesses to prevent numerical issues
    scale = np.max(np.abs(x0)) + 1e-8
    
    # Define a scaled version of the original function to handle scale differences in variables
    def scaled_fun(x, *args):
        return fun(x * scale, *args)
    
    # If a Jacobian function is provided, scale it appropriately to correspond with the scaled function
    def scaled_jac(x, *args):
        return jac(x * scale, *args) * scale
    
    # Perform minimization on the scaled function and adjust the initial guess and Jacobian accordingly
    result = minimize(scaled_fun, x0 / scale, args=args, jac=scaled_jac if jac else None, **kwargs)
    
    # Adjust the result to reflect the original scale and return the scaled optimization result
    return OptimizeResult(
        x=result.x * scale,  # Scale the solution back to the original scale
        fun=result.fun,  # The function value at the solution
        jac=result.jac / scale if result.jac is not None else None,  # Scale the Jacobian back if it exists
        **{k: v for k, v in result.items() if k not in ['x', 'fun', 'jac']}  # Pass through other info unchanged
    )

In [227]:
# Setting up the path to the TNTP data
networks_path = Path("./TransportationNetworks")
folder = "SiouxFalls"
net_name = "SiouxFalls_net"
traffic_mat_name = "SiouxFalls_trips"

# Loading graph and traffic data
net_file = networks_path / folder / f"{net_name}.tntp"
traffic_mat_file = networks_path / folder / f"{traffic_mat_name}.tntp"
graph, metadata = read_graph_transport_networks_tntp(net_file)
correspondences = read_traffic_mat_transport_networks_tntp(traffic_mat_file, metadata)

beckmann_model = BeckmannModel(graph, correspondences)

# Retrieving the number of nodes and edges from the graph
nodes = graph.number_of_nodes()
edges = graph.number_of_edges()

metadata["can_pass_through_zones"]=True


In [228]:
# Generate the incidence matrix for the city model
A = nx.incidence_matrix(graph, oriented=True).todense()
print("Incidence matrix (A) shape:", A.shape)
print("Incidence matrix (A) sample:\n", A[:5, :5])  # First 5 rows and columns

# Using the traffic matrix as the demand matrix L
correspondence_matrix = np.array(correspondences.traffic_mat)  # Ensure it's in NumPy array form if not already
L = np.diag(correspondence_matrix.sum(axis=1)) - correspondence_matrix.T

print("Demand matrix (L) shape:", L.shape)
print("Demand matrix (L) sample:\n", L[:5, :5])  # First 5 rows and columns

beta = 0.1
y_init = np.random.rand(nodes, nodes).flatten() * 1e-3

options = {
    'maxiter': 1000,
    'disp': True,
    'ftol': 1e-6,
    'gtol': 1e-6
}

# Perform the optimization using L-BFGS-B
try:
    result = minimize(dual_objective, y_init, jac=dual_objective_grad, args=(A, L, beta), method='L-BFGS-B', options=options)
    print("Optimization Result:", result)
except Exception as e:
    print(f"An error occurred during optimization: {e}")

print("Optimization Result:", result)

Incidence matrix (A) shape: (24, 76)
Incidence matrix (A) sample:
 [[-1. -1.  1.  0.  1.]
 [ 1.  0. -1. -1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
Demand matrix (L) shape: (24, 24)
Demand matrix (L) sample:
 [[ 8800.  -100.  -100.  -500.  -200.]
 [ -100.  4000.  -100.  -200.  -100.]
 [ -100.  -100.  2800.  -200.  -100.]
 [ -500.  -200.  -200. 11600.  -500.]
 [ -200.  -100.  -100.  -500.  6100.]]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          576     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.68300D+07    |proj g|=  4.51997D+04

At iterate    1    f=  4.51569D+07    |proj g|=  3.17174D+04

At iterate    2    f=  4.34603D+07    |proj g|=  5.92010D+04

At iterate    3    f=  4.17771D+07    |proj g|=  9.54504D+03

At iterate    4    f=  4.15237D+07    |proj g|=  4.55104D+03

At iterate    5    f=  4.08495D+07    |proj g|=  9.07431D+03

At iterate    6    f=  4.

 This problem is unconstrained.



At iterate   14    f=  4.00937D+07    |proj g|=  2.71689D+03

At iterate   15    f=  4.00823D+07    |proj g|=  1.07990D+03

At iterate   16    f=  4.00784D+07    |proj g|=  4.74352D+02

At iterate   17    f=  4.00729D+07    |proj g|=  3.12839D+02

At iterate   18    f=  4.00698D+07    |proj g|=  1.49431D+03

At iterate   19    f=  4.00672D+07    |proj g|=  3.49205D+02

At iterate   20    f=  4.00662D+07    |proj g|=  1.45281D+02

At iterate   21    f=  4.00655D+07    |proj g|=  4.29335D+02

At iterate   22    f=  4.00652D+07    |proj g|=  7.30396D+02

At iterate   23    f=  4.00647D+07    |proj g|=  9.69463D+01

At iterate   24    f=  4.00645D+07    |proj g|=  9.36916D+01

At iterate   25    f=  4.00643D+07    |proj g|=  1.60772D+02

At iterate   26    f=  4.00642D+07    |proj g|=  2.96765D+02

At iterate   27    f=  4.00641D+07    |proj g|=  1.38809D+02

At iterate   28    f=  4.00640D+07    |proj g|=  2.80719D+01
Optimization Result:   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FAC